# Data Processing

## Import Libraries

In [1]:
# import library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange
import pickle as pkl
from PIL import Image

# torch
import torch
from torch import nn
from torch.nn import Module, Linear, Dropout
import torch.nn.functional as F
from torch.optim import Adam, SGD

# import custom functions
from utils import load_zipped_pickle, save_zipped_pickle

DATA_DIR = "Data"

## Load Data

In [2]:
train_data = load_zipped_pickle(os.path.join(DATA_DIR, "train.pkl"))
test_data = load_zipped_pickle(os.path.join(DATA_DIR, "test.pkl"))
samples = load_zipped_pickle(os.path.join(DATA_DIR, "sample.pkl"))

## Preprocess Data

In [3]:
train_data[0]["video"].shape

(112, 112, 334)

In [4]:
list(train_data[0].keys())

['name', 'video', 'box', 'label', 'frames', 'dataset']

In [5]:
# 低分辨率的数据不需要进一步处理，直接提取有标签的数据。
# 高分辨率的数据需要进行切分，切分为一个个112*112的子图形和标签
def tailor_img(img: np.array, target_size: tuple = (112, 112), stripe: int = 2) -> [np.array]:
    tailored_imgs = []
    for i in range(0, img.shape[0] - target_size[0] + 1, stripe):
        for j in range(0, img.shape[1] - target_size[1] + 1, stripe):
            sub_img = img[i:i + target_size[0], j:j + target_size[1]]
            tailored_imgs.append(sub_img)
    return tailored_imgs

low_resolution_data = [x for x in train_data if x["video"].shape[0] == 112 and x["video"].shape[1] == 112]
high_resolution_data = [x for x in train_data if x["video"].shape[0] != 112 or x["video"].shape[1] != 112]

# 将低分辨率数据转换为训练所用的(帧数，高，宽)数据
low_resolution_x = np.concatenate([x["video"][:, :, x["frames"]].transpose((2, 0, 1)) for x in low_resolution_data], axis=0) # 将shape转换为 (帧数，高，宽)
low_resolution_y = np.concatenate([y["label"][:, :, y["frames"]].transpose((2, 0, 1)) for y in low_resolution_data], axis=0)

# 对高分辨数据进行裁切，并转为训练所用的(帧数，高，宽)数据
high_resolution_x = []
high_resolution_y = []
for video in high_resolution_data:
    images_x, images_y = video["video"][:, :, video["frames"]], video["label"][:, :, video["frames"]]
    images_x = images_x.transpose((2, 0, 1)) # 将shape转换为 (帧数，高，宽)
    images_y = images_y.transpose((2, 0, 1))
    for image_x, image_y in zip(images_x, images_y):
        tailored_imgs_x = tailor_img(image_x, target_size=(112, 112), stripe=3)
        tailored_imgs_y = tailor_img(image_y, target_size=(112, 112), stripe=3)

        high_resolution_x += tailored_imgs_x
        high_resolution_y += tailored_imgs_y

In [48]:
# 将低维数据输出位png图片
# image_array是归一化的二维浮点数矩阵
img_x = low_resolution_x[1]
img_y = low_resolution_y[1]
img_x = Image.fromarray(img_x)
img_y = Image.fromarray(img_y)
img_x = img_x.convert('L')  # 这样才能转为灰度图，如果是彩色图则改L为‘RGB’
img_y = img_y.convert('L')  # 这样才能转为灰度图，如果是彩色图则改L为‘RGB’
img_x.save('./pictures/low_resolution_x.png')
img_y.save('./pictures/low_resolution_y.png')

In [28]:
# 将高维数据输出为png图片
# image_array是归一化的二维浮点数矩阵
img_x = high_resolution_x[0]
img_y = high_resolution_x[0]
img_x = Image.fromarray(img_x)
img_y = Image.fromarray(img_y)
img_x = img_x.convert('L')  # 这样才能转为灰度图，如果是彩色图则改L为‘RGB’
img_y = img_y.convert('L')  # 这样才能转为灰度图，如果是彩色图则改L为‘RGB’
img_x.save('./pictures/high_resolution_x.png')
img_y.save('./pictures/high_resolution_y.png')

In [6]:
# 筛选高分辨数据中有效的数据
valid_ids = []
high_resolution_x_new = []
high_resolution_y_new = []
for i, (x, y) in enumerate(zip(high_resolution_x, high_resolution_y)):
    if y.sum() > 20:
        valid_ids.append(i)
        high_resolution_x_new.append(x)
        high_resolution_y_new.append(y)
high_resolution_x = np.array(high_resolution_x_new)
high_resolution_y = np.array(high_resolution_y_new)
# 保存训练数据
np.save("./Data/high_resolution_x.npy", high_resolution_x)
np.save("./Data/high_resolution_y.npy", high_resolution_y)